In [1]:
from datasets import load_dataset

In [2]:
'''import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch'''

'import os\nos.environ["CUDA_VISIBLE_DEVICES"] = "1"\nimport torch'

In [3]:
food = load_dataset("food101", split="train[:5000]")

In [4]:
food = food.train_test_split(test_size=0.2)

In [5]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [6]:
id2label[str(79)]

'prime_rib'

In [7]:
from transformers import AutoImageProcessor

In [8]:
checkpoint = "google/vit-base-patch16-224-in21k"

In [9]:
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [10]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

In [11]:
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)

In [12]:
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])

)

_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [13]:
def transforms(examples):

    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]

    del examples["image"]

    return examples

In [14]:
food = food.with_transform(transforms)

In [15]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
import numpy as np


def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(

    checkpoint,

    num_labels=len(labels),

    id2label=id2label,

    label2id=label2id,

)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:

training_args = TrainingArguments(

    output_dir="my_awesome_food_model",

    remove_unused_columns=False,

    evaluation_strategy="epoch",

    save_strategy="epoch",

    learning_rate=5e-5,

    per_device_train_batch_size=8,

    gradient_accumulation_steps=4,

    per_device_eval_batch_size=8,

    num_train_epochs=3,

    warmup_ratio=0.1,

    logging_steps=10,

    load_best_model_at_end=True,

    metric_for_best_model="accuracy",

    push_to_hub=False,


)




# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

/home/guslasbo@GU.GU.SE/miniconda3/envs/Assignment1_ML/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
0,3.520900,3.289813,0.759000
1,2.687800,2.591618,0.823000
2,2.384400,2.412819,0.825000


/home/guslasbo@GU.GU.SE/miniconda3/envs/Assignment1_ML/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/guslasbo@GU.GU.SE/miniconda3/envs/Assignment1_ML/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=93, training_loss=3.111316450180546, metrics={'train_runtime': 525.3565, 'train_samples_per_second': 22.842, 'train_steps_per_second': 0.177, 'total_flos': 9.232831524962304e+17, 'train_loss': 3.111316450180546, 'epoch': 2.98})

In [20]:
ds = load_dataset("food101", split="validation[:10]")

image = ds["image"][0]

In [21]:
!nvidia-smi

Tue Oct 31 18:44:46 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off | 00000000:03:00.0 Off |                  N/A |
| 86%   85C    P2              70W / 250W |   2946MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [30]:
from transformers import pipeline

classifier = pipeline("image-classification", model="./my_awesome_food_model/checkpoint-93")

classifier(image)


[{'score': 0.13332407176494598, 'label': 'beignets'},
 {'score': 0.02000979147851467, 'label': 'prime_rib'},
 {'score': 0.01762833632528782, 'label': 'chicken_wings'},
 {'score': 0.017359808087348938, 'label': 'bruschetta'},
 {'score': 0.014923209324479103, 'label': 'hamburger'}]

In [26]:
from transformers import AutoImageProcessor

import torch

image_processor = AutoImageProcessor.from_pretrained("./my_awesome_food_model/checkpoint-93")

inputs = image_processor(image, return_tensors="pt")

In [27]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained("./my_awesome_food_model/checkpoint-93")

with torch.no_grad():

    logits = model(**inputs).logits

In [29]:
predicted_label = logits.argmax(-1).item()

model.config.id2label[predicted_label]


'beignets'